In [ ]:
# Cell A — install
!pip -q install chromadb "sentence-transformers>=3.0" scikit-learn tqdm regex openai tiktoken

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "put your key here"

from openai import OpenAI
oai = OpenAI()

In [8]:
# Cell B — config & load CSV
import os, json, uuid, re
from pathlib import Path
import pandas as pd
import numpy as np
from tqdm import tqdm

# Expect OPENAI_API_KEY in your environment (don't hardcode keys)
assert os.getenv("OPENAI_API_KEY"), "Set OPENAI_API_KEY in your environment before running."

CSV = Path.home() / "Documents" / "UChicago" / "Qrt 4" / "Gen AI" / "Midterm" / "Text" / "chunks.csv"
df = pd.read_csv(CSV)

# basic cleaning
df['text'] = df['text'].fillna('').astype(str).str.strip()
df = df[df['text'].str.len() > 180]
df = df.drop_duplicates(subset=['text'])

for col in ['id','url','title','section','type','date']:
    if col not in df.columns:
        df[col] = ''
df.reset_index(drop=True, inplace=True)

len(df), df.head(2)

(1150,
                 id                                      url        title  \
 0  3e4c2d399c-0000  https://datascience.uchicago.edu/about/  About - DSI   
 1  3e4c2d399c-0001  https://datascience.uchicago.edu/about/  About - DSI   
 
   section     type  date  chunk_id  \
 0   about  generic   NaN         0   
 1   about  generic   NaN         1   
 
                                                 text  \
 0  The UChicago Data Science Institute executes t...   
 1  The Data Science Institute is located on the U...   
 
                                               sha256  
 0  93a410375295f383d3323f1bfcfedee3f6c2c05f0b4bd5...  
 1  5af0feb9df7cd0add84cd7ea4f5a0f9826fb553d1031ec...  )

In [10]:
# Cell C — embeddings + Chroma setup
import chromadb
from chromadb.utils import embedding_functions
from sentence_transformers import SentenceTransformer

EMBED_MODEL_NAME = "intfloat/e5-base-v2"   # you can upgrade to e5-large-v2 later
model = SentenceTransformer(EMBED_MODEL_NAME)

class E5Embedder(embedding_functions.EmbeddingFunction):
    def __call__(self, input: list[str]) -> list[list[float]]:
        v = model.encode(["passage: " + x for x in input],
                         normalize_embeddings=True, convert_to_numpy=True)
        return v.tolist()

CHROMA_DIR = Path("rag_index") / "chroma_db"
CHROMA_DIR.mkdir(parents=True, exist_ok=True)
client = chromadb.PersistentClient(path=str(CHROMA_DIR))

collection = client.get_or_create_collection(
    name="msads_e5",
    metadata={"hnsw:space":"cosine"},
    embedding_function=E5Embedder()
)


/var/folders/59/_xzwcqns6vj35vgt26hxm7qh0000gn/T/ipykernel_82012/3416729326.py:22: DeprecationWarning: The class E5Embedder does not implement __init__. This will be required in a future version.
  embedding_function=E5Embedder()


In [14]:
# Cell D — build/refresh Chroma from df
docs = df["text"].tolist()
doc_ids = df["id"].astype(str).tolist() if "id" in df.columns and df["id"].notna().all() else [str(uuid.uuid4()) for _ in range(len(df))]
metas = []
for _, r in df.iterrows():
    metas.append({
        "url":     str(r.get("url","")),
        "title":   str(r.get("title","")),
        "section": str(r.get("section","")),
        "type":    str(r.get("type","")),
        "date":    str(r.get("date","")),
    })

# Optional clean rebuild:
# client.delete_collection("msads_e5")
# collection = client.create_collection(name="msads_e5", metadata={"hnsw:space":"cosine"}, embedding_function=E5Embedder())

# Upsert in batches
B=256
for i in range(0, len(docs), B):
    collection.add(ids=doc_ids[i:i+B], documents=docs[i:i+B], metadatas=metas[i:i+B])

print("Chroma doc count:", collection.count())


Chroma doc count: 1150


In [17]:
# Cell E — TF-IDF corpus for hybrid
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

META = []
id_to_meta = {}
id_order = []  # aligns with corpus rows

for i in range(len(docs)):
    m = metas[i].copy()
    m["text"] = docs[i]
    m["_id"]  = doc_ids[i]
    META.append(m)
    id_to_meta[doc_ids[i]] = m
    id_order.append(doc_ids[i])

DOC_TEXTS = [
    (m.get("title","") + " " + m.get("section","") + " " + m.get("url","") + " " + m.get("text","")).strip()
    for m in META
]

tfidf = TfidfVectorizer(max_df=0.9, min_df=2, ngram_range=(1,2))
X = tfidf.fit_transform(DOC_TEXTS)

def bm25_like_indices(query, topn=60):
    qv = tfidf.transform([query])
    sims = cosine_similarity(qv, X).ravel()
    idx = np.argsort(-sims)[:topn]
    return idx, sims[idx]


In [19]:
# Cell F — hybrid retrieval with MMR + optional cross-encoder rerank
try:
    from sentence_transformers import CrossEncoder
    reranker = CrossEncoder("BAAI/bge-reranker-base")
except Exception:
    reranker = None

EDU_PRIORITY_HINTS = [
    "/education/masters-programs/ms-in-applied-data-science",
    "/education/masters-programs",
    "/education/",
]
LOW_PRIORITY_HINTS = [
    "/news-events/news/",
    "/news-events/events/",
    "/news-events/insights/",
    "/research/",
    "/people/",
]
INTENT_KEYWORDS = {
    "admissions": ["admission","admissions","apply","application","requirements","prereq","prerequisite","deadline","GRE","TOEFL","IELTS","resume","statement","letters"],
    "curriculum": ["core","course","courses","curriculum","credit","unit","track","specialization","elective"],
    "capstone":   ["capstone","project","showcase","practicum"],
}

def _intent(query: str):
    ql = query.lower()
    for k, toks in INTENT_KEYWORDS.items():
        if any(t in ql for t in toks):
            return k
    return None

def _boost_score(url: str, section: str, base: float) -> float:
    b = base
    if any(h in url for h in EDU_PRIORITY_HINTS):  b += 0.20
    if section == "education":                     b += 0.10
    if any(h in url for h in LOW_PRIORITY_HINTS):  b -= 0.20
    return b

def ann_dense_chroma(query: str, topn=60):
    qvec = model.encode(["query: " + query], normalize_embeddings=True, convert_to_numpy=True)[0].tolist()
    res = collection.query(query_embeddings=[qvec], n_results=topn, include=["metadatas","documents","distances","ids"])
    ids = res["ids"][0]
    sims = [1.0 - d for d in res["distances"][0]]  # cosine similarity
    return ids, sims

def mmr_select(q_vec: np.ndarray, cand_vecs: np.ndarray, cand_ids: list[str], k: int = 6, lambda_: float = 0.55):
    selected, pool = [], list(range(len(cand_ids)))
    sim_q = (cand_vecs @ q_vec.reshape(-1,1)).ravel()
    while len(selected) < min(k, len(pool)):
        if not selected:
            i = int(np.argmax(sim_q[pool])); selected.append(pool[i]); pool.pop(i); continue
        sel_vecs = cand_vecs[selected]
        cand_idx = np.array(pool)
        diversity = (cand_vecs[cand_idx] @ sel_vecs.T).max(axis=1)
        scores = lambda_ * sim_q[cand_idx] - (1 - lambda_) * diversity
        i = int(np.argmax(scores))
        selected.append(cand_idx[i])
        pool.remove(cand_idx[i])
    return [cand_ids[i] for i in selected]

def retrieve_hybrid(query: str, k=6, shortlist=60, use_reranker=True):
    # dense (Chroma)
    dense_ids, _ = ann_dense_chroma(query, topn=shortlist)
    # sparse (TF-IDF)
    sparse_idx, _ = bm25_like_indices(query, topn=shortlist)
    sparse_ids = [id_order[i] for i in sparse_idx]

    # RRF
    def rrf(id_lists, c=60):
        score = {}
        for lst in id_lists:
            for r, did in enumerate(lst):
                score[did] = score.get(did, 0.0) + 1.0/(c + r + 1)
        return score
    fused = rrf([list(dense_ids), list(sparse_ids)])

    # boosts + intent gating
    want = _intent(query)
    items = []
    for did, base in fused.items():
        m = id_to_meta[did]
        url = m.get("url",""); sec = m.get("section","")
        if want not in ("capstone",) and ("/news-events/" in url or "/research/" in url):
            base -= 0.25
        items.append((did, _boost_score(url, sec, base)))
    items.sort(key=lambda x: -x[1])

    # MMR on a larger pool
    pool = [did for did,_ in items[:max(k, 30)]]
    cand_texts = [id_to_meta[did]["text"] for did in pool]
    cand_vecs  = model.encode(["passage: " + t for t in cand_texts], normalize_embeddings=True, convert_to_numpy=True)
    q_vec      = model.encode(["query: " + query], normalize_embeddings=True, convert_to_numpy=True).ravel().astype(np.float32)
    mmr_ids    = mmr_select(q_vec, cand_vecs, pool, k=max(k, 10), lambda_=0.55)

    hits = [dict(id_to_meta[did]) | {"_id": did} for did in mmr_ids]

    # optional rerank
    if use_reranker and reranker is not None:
        pairs  = [(query, h["text"]) for h in hits]
        scores = reranker.predict(pairs)
        for h, s in zip(hits, scores):
            h["rerank_score"] = float(s)
        hits.sort(key=lambda x: -x["rerank_score"])

    return hits[:k]

# this overrides any earlier retriever
def retrieve(query, k=6, use_reranker=True):
    return retrieve_hybrid(query, k=k, shortlist=60, use_reranker=use_reranker)


In [20]:
# Cell G — compression + context with citations
PII_EMAIL = re.compile(r'[\w\.-]+@[\w\.-]+\.\w+')
PII_PHONE = re.compile(r'\b(?:\+?\d{1,2}\s*)?(?:\(?\d{3}\)?[\s.-]*)?\d{3}[\s.-]?\d{4}\b')
_SENT_SPLIT = re.compile(r'(?<=[\.\?!])\s+(?=[A-Z0-9])')

def scrub(text: str) -> str:
    text = PII_EMAIL.sub("[redacted-email]", text)
    text = PII_PHONE.sub("[redacted-phone]", text)
    return text

def compress_text_for_query(text: str, query: str, top_sentences: int = 8):
    sents = [s.strip() for s in _SENT_SPLIT.split((text or "").strip()) if s.strip()]
    if not sents: return text or ""
    qv = model.encode(["query: " + query], normalize_embeddings=True, convert_to_numpy=True)
    sv = model.encode(["passage: " + s for s in sents], normalize_embeddings=True, convert_to_numpy=True)
    sims = (sv @ qv.T).ravel()
    keep = np.argsort(-sims)[:min(top_sentences, len(sents))]
    keep.sort()
    return " ".join(sents[i] for i in keep)

def long_context_reorder(hits):
    if len(hits) <= 2: return hits
    L, R, out = 0, len(hits)-1, []
    while L <= R:
        out.append(hits[L]); L += 1
        if L <= R: out.append(hits[R]); R -= 1
    return out

def build_context(hits, query):
    hits = long_context_reorder(hits)
    blocks = []
    for i, h in enumerate(hits, 1):
        title = (h.get('title','') or '').strip()
        url   = (h.get('url','') or '').strip()
        sect  = (h.get('section','') or '').strip()
        txt   = compress_text_for_query(h.get('text','') or '', query, top_sentences=8)
        txt   = scrub(txt)
        blocks.append(f"[{i}] {title} • {sect} | {url}\n{txt}")
    return "\n\n---\n\n".join(blocks)


In [80]:
# Cell H — OpenAI answerer + ask() helper
from openai import OpenAI
oai = OpenAI()

SYSTEM_PROMPT = """You are a helpful assistant for the University of Chicago MS in Applied Data Science.
Answer ONLY from the provided context. Prefer content from the Education section and the program page.
If top results are news, events, insights, research, or people pages, treat them as lower priority unless the question asks for them.
If the required information is not present in the provided context, say you don't know and suggest checking the official MS-ADS page.
Keep answers specific and concise. Always include bracketed citations like [1], [2] with URLs.
Redact personal emails/phones if present in context.
"""

def answer(query, k=6, use_reranker=True, model_name="gpt-4o-mini", temperature=0.2):
    hits = retrieve(query, k=k, use_reranker=use_reranker)
    if not any(("/education/" in (h.get("url","") or "") or "ms-in-applied-data-science" in (h.get("url","") or "")) for h in hits):
        q_aug = query + ' program site education admissions curriculum "MS in Applied Data Science"'
        hits = retrieve(q_aug, k=k, use_reranker=use_reranker)

    context = build_context(hits, query=query)
    user_prompt = f"Question: {query}\n\nUse the context to answer with bracket citations.\n\nContext:\n{context}"

    resp = oai.chat.completions.create(
        model=model_name,
        temperature=temperature,
        messages=[
            {"role":"system","content":SYSTEM_PROMPT},
            {"role":"user","content":user_prompt}
        ]
    )
    text = resp.choices[0].message.content.strip()

    # === Deduplicate sources before returning ===
    unique_sources = []
    seen_urls = set()
    for i, h in enumerate(hits, 1):
        url = h.get('url', '') or ''
        title = h.get('title', '(no title)')
        if url in seen_urls:
            continue
        seen_urls.add(url)
        unique_sources.append(f"[{len(unique_sources)+1}] {title} — {url}")

    refs_block = "\n\n📚 References\n" + "\n".join(unique_sources)
    text = text.split("\n\nSources:")[0].strip() + refs_block
    # ================================================
    
    return text, hits

def ask(q: str, k: int = 6):
    print("Q:", q, "\n" + "-"*80)
    ans, hits = answer(q, k=k, use_reranker=True)
    print(ans, "\n")
    for i, h in enumerate(hits, 1):
        print(f"[{i}] {h.get('title','')} — {h.get('url','')}")
    return ans, hits


In [31]:
# PATCH: redefine ann_dense_chroma without "ids" in include
def ann_dense_chroma(query: str, topn=60):
    # embed the query with E5 "query:" prefix
    qvec = model.encode(["query: " + query],
                        normalize_embeddings=True,
                        convert_to_numpy=True)[0].tolist()
    # NOTE: do NOT include "ids" here — Chroma always returns ids
    res = collection.query(
        query_embeddings=[qvec],
        n_results=topn,
        include=["metadatas", "documents", "distances"]  # no "ids"
    )
    ids = res["ids"][0]                      # still present by default
    sims = [1.0 - d for d in res["distances"][0]]  # cosine similarity
    return ids, sims


In [35]:
ask("What are the admission requirements for the MS in Applied Data Science program?")

Q: What are the admission requirements for the MS in Applied Data Science program? 
--------------------------------------------------------------------------------
The admission requirements for the MS in Applied Data Science program at the University of Chicago include the following:

1. A completed application form.
2. Academic transcripts from all post-secondary institutions attended.
3. A statement of purpose.
4. A resume or CV.
5. Letters of recommendation.
6. GRE scores may be required or optional depending on the applicant's background.

For more detailed information, it is recommended to check the official MS-ADS page [3]. 

[1] Should You Choose MS in Data Science or MS in Applied Data Science at UChicago? — https://datascience.uchicago.edu/news/should-you-choose-ms-in-data-science-or-ms-in-applied-data-science-at-uchicago/
[2] Master's in Applied Data Science - DSI — https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/
[3] Should You Choose

("The admission requirements for the MS in Applied Data Science program at the University of Chicago include the following:\n\n1. A completed application form.\n2. Academic transcripts from all post-secondary institutions attended.\n3. A statement of purpose.\n4. A resume or CV.\n5. Letters of recommendation.\n6. GRE scores may be required or optional depending on the applicant's background.\n\nFor more detailed information, it is recommended to check the official MS-ADS page [3].",
 [{'url': 'https://datascience.uchicago.edu/news/should-you-choose-ms-in-data-science-or-ms-in-applied-data-science-at-uchicago/',
   'title': 'Should You Choose MS in Data Science or MS in Applied Data Science at UChicago?',
   'section': 'news',
   'type': 'generic',
   'date': 'nan',
   'text': 'Choose the MS in Data Science if you: Are passionate about understanding the theoretical foundations of data science Want to pursue research alongside world-class faculty Are considering a PhD or research-focused

In [43]:
ask("Can you provide information about the capstone project? Keep it detailed and tell me what is expected of students?")

Q: Can you provide information about the capstone project? Keep it detailed and tell me what is expected of students? 
--------------------------------------------------------------------------------
The capstone project in the University of Chicago's MS in Applied Data Science program is a significant component where students apply their learning to real-world problems. Each student or team collaborates with industry partners to develop a project that demonstrates their ability to innovate and drive meaningful impact using applied data science techniques.

Students are expected to engage in the following activities during their capstone project:

1. **Problem Identification**: Students must identify a relevant problem within their chosen industry or field that can be addressed through data science methodologies.

2. **Data Collection and Analysis**: They are required to gather and analyze data, which may involve scraping, standardizing, and aggregating data from various sources, as de

("The capstone project in the University of Chicago's MS in Applied Data Science program is a significant component where students apply their learning to real-world problems. Each student or team collaborates with industry partners to develop a project that demonstrates their ability to innovate and drive meaningful impact using applied data science techniques.\n\nStudents are expected to engage in the following activities during their capstone project:\n\n1. **Problem Identification**: Students must identify a relevant problem within their chosen industry or field that can be addressed through data science methodologies.\n\n2. **Data Collection and Analysis**: They are required to gather and analyze data, which may involve scraping, standardizing, and aggregating data from various sources, as demonstrated in past projects.\n\n3. **Application of Techniques**: Students apply advanced data science techniques, such as time-series forecasting, machine learning, or data visualization, to 

In [40]:
ask("What are the core courses in the MS in Applied Data Science program?")

Q: What are the core courses in the MS in Applied Data Science program? 
--------------------------------------------------------------------------------
The core courses in the MS in Applied Data Science program include a focus on statistical modeling, machine learning, data engineering, cloud computing, and AI principles. The program is designed to provide hands-on expertise across the full spectrum of data science, preparing students to be job-ready and make an immediate impact across various industries [3][4]. 

For more detailed information, you may want to check the official MS-ADS page. 

[1] Undergraduate Data Science Major — https://datascience.uchicago.edu/education/undergrad-major/
[2] Should You Choose MS in Data Science or MS in Applied Data Science at UChicago? — https://datascience.uchicago.edu/news/should-you-choose-ms-in-data-science-or-ms-in-applied-data-science-at-uchicago/
[3] Info Session – MS in Applied Data Science (In-Person Program) — https://datascience.uchica

('The core courses in the MS in Applied Data Science program include a focus on statistical modeling, machine learning, data engineering, cloud computing, and AI principles. The program is designed to provide hands-on expertise across the full spectrum of data science, preparing students to be job-ready and make an immediate impact across various industries [3][4]. \n\nFor more detailed information, you may want to check the official MS-ADS page.',
 [{'url': 'https://datascience.uchicago.edu/education/undergrad-major/',
   'title': 'Undergraduate Data Science Major',
   'section': 'education',
   'type': 'generic',
   'date': 'nan',
   'text': 'The technological revolution has led to an explosion of data in domains of knowledge including medicine, policy, social sciences, commerce, and the natural sciences. Petabytes of data are being collected from a myriad of instruments, like sequencing machines for genomics and mobile devices for quantifying social interactions. In addition to driv

In [61]:
ask(" who are the faculties for MS in Applied data Science? Give me list ")

Q:  who are the faculties for MS in Applied data Science? Give me list  
--------------------------------------------------------------------------------
The faculty for the MS in Applied Data Science program includes:

1. Arnab Bose - Associate Senior Instructional Professor, Program Director of MS in Applied Data Science Online Program
2. Nick Ross - Data Science Clinic Director, Data Science Institute; Associate Senior Instructional Professor
3. Jesse London - Professor, Booth School of Business
4. Susan Paykin - Software Engineer, Data Science Institute
5. Taylor Alexander - MA, Associate Director, Instructional Services, MS in Applied Data Science
6. Briana Allen - MEd, Director, Academic Affairs and Operations
7. Jose Alvarado - Associate Director, Enrollment Management, MS in Applied Data Science
8. Josie Badillo Sittig - Assistant Director, Marketing/Advertising & Communications, MS in Applied Data Science
9. Nick DiAntonio - MEd, Graduate Academic Advisor
10. Lauren Isaacman D

('The faculty for the MS in Applied Data Science program includes:\n\n1. Arnab Bose - Associate Senior Instructional Professor, Program Director of MS in Applied Data Science Online Program\n2. Nick Ross - Data Science Clinic Director, Data Science Institute; Associate Senior Instructional Professor\n3. Jesse London - Professor, Booth School of Business\n4. Susan Paykin - Software Engineer, Data Science Institute\n5. Taylor Alexander - MA, Associate Director, Instructional Services, MS in Applied Data Science\n6. Briana Allen - MEd, Director, Academic Affairs and Operations\n7. Jose Alvarado - Associate Director, Enrollment Management, MS in Applied Data Science\n8. Josie Badillo Sittig - Assistant Director, Marketing/Advertising & Communications, MS in Applied Data Science\n9. Nick DiAntonio - MEd, Graduate Academic Advisor\n10. Lauren Isaacman Darga - MA, Assistant Director, External Partnerships, MS in Applied Data Science\n11. Emma Kerr-Ketchum - MA, Assistant Director, Instruction

In [56]:
ask("Are there opportunities to work with real-world datasets or industry partners?")

Q: Are there opportunities to work with real-world datasets or industry partners? 
--------------------------------------------------------------------------------
Yes, there are opportunities to work with real-world datasets and industry partners in the University of Chicago's MS in Applied Data Science program. The program includes a real-world Capstone Experience where students collaborate with top companies to solve actual business problems. Additionally, the Data Science Clinic allows students to work on projects with real-world clients, partnering with various organizations for hands-on experience in data science projects [1][3]. 

[1] Master's in Applied Data Science - DSI — https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/
[2] Data Science Clinic — https://datascience.uchicago.edu/education/data-science-clinic/
[3] Master’s in Applied Data Science Summer 2025 Capstone Showcase — https://datascience.uchicago.edu/news/masters-in-applied-data-

("Yes, there are opportunities to work with real-world datasets and industry partners in the University of Chicago's MS in Applied Data Science program. The program includes a real-world Capstone Experience where students collaborate with top companies to solve actual business problems. Additionally, the Data Science Clinic allows students to work on projects with real-world clients, partnering with various organizations for hands-on experience in data science projects [1][3].",
 [{'url': 'https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/',
   'title': "Master's in Applied Data Science - DSI",
   'section': 'education',
   'type': 'generic',
   'date': 'nan',
   'text': 'The University of Chicago’s MS in Applied Data Science program equips you with in-demand expertise and an unparalleled network of global alumni. Take the next step and start your application today. Choose from full- and part-time options in our In-Person and Online programs. Apply 

In [64]:
# F-1 — intent, allowlist/denylist, query rewrite for faculty
FACULTY_INTENT_TOKENS = [
    "faculty","faculties","instructor","instructors","teaching team","teaching faculty",
    "staff","instructors & staff","instructors and staff"
]

def is_faculty_intent(q: str) -> bool:
    ql = q.lower()
    return any(t in ql for t in FACULTY_INTENT_TOKENS)

# Pages we want to prioritize or restrict to
FACULTY_ALLOW = [
    "/education/masters-programs/ms-in-applied-data-science/instructors-staff/",
]
FACULTY_STRONG_HINTS = [
    "/education/masters-programs/ms-in-applied-data-science",
    "/education/masters-programs",
    "/education/",
]
FACULTY_DENY = [
    "/education/undergrad", "/education/phd-in-data-science", "/undergrad", "/phd-in-data-science",
    "/news-events/", "/research/", "/people/"  # People directory is too broad for this query
]

def faculty_query_rewrite(q: str) -> str:
    # steer retrieval toward the right page without changing your index
    extras = ' "instructors & staff" "MS in Applied Data Science" instructors staff program'
    return q + " " + extras


In [66]:
# F-2 — retrieve_faculty(): use your hybrid retriever, then filter/boost to MS-ADS instructors page
def _faculty_score_url(url: str, base: float = 0.0) -> float:
    sc = base
    if any(h in url for h in FACULTY_ALLOW):        sc += 0.80   # hard boost for the exact page
    if any(h in url for h in FACULTY_STRONG_HINTS): sc += 0.20
    if any(h in url for h in FACULTY_DENY):         sc -= 0.60
    return sc

def retrieve_faculty(query: str, k: int = 6, shortlist: int = 30):
    # 1) use your hybrid retriever but with the rewritten query and a bigger shortlist
    q2 = faculty_query_rewrite(query)
    # NOTE: call the underlying hybrid with a larger shortlist by temporarily exposing it if needed.
    # If retrieve_hybrid isn't directly accessible with shortlist, we approximate by calling twice:
    hits = retrieve_hybrid(q2, k=max(k, 12), shortlist=60, use_reranker=True)

    # 2) score with URL heuristics and keep the best
    scored = []
    for h in hits:
        u = h.get("url","") or ""
        scored.append((_faculty_score_url(u, base=0.0), h))
    scored.sort(key=lambda x: -x[0])

    # 3) if the exact instructors-staff page is NOT in the list, do a second pass from the whole META:
    if not any(FACULTY_ALLOW[0] in (h.get("url","") or "") for _, h in scored):
        # brute-force pull candidates from META whose URL looks relevant
        cand = [m for m in META if (FACULTY_ALLOW[0] in m.get("url","")) or
                                  ("/ms-in-applied-data-science" in m.get("url","") and "/education/" in m.get("url",""))]
        if cand:
            # rank those candidates by embed similarity to the rewritten query
            qv = model.encode(["query: " + q2], normalize_embeddings=True, convert_to_numpy=True)
            sv = model.encode(["passage: " + c["text"] for c in cand], normalize_embeddings=True, convert_to_numpy=True)
            sims = (sv @ qv.T).ravel()
            order = np.argsort(-sims)
            topc = [cand[i] for i in order[:max(k,3)]]
            # put these at the front
            scored = [(_faculty_score_url(c.get("url",""), base=1.0), c) for c in topc] + scored

    # 4) finalize top-k
    final = []
    seen_urls = set()
    for sc, h in scored:
        u = h.get("url","") or ""
        if any(d in u for d in FACULTY_DENY): 
            continue
        if u in seen_urls:
            continue
        seen_urls.add(u)
        final.append(h)
        if len(final) >= k:
            break
    return final


In [68]:
# F-3 — extract roster lines "Name — Title" from page text (bullets or paragraphs)
NAME_LINE = re.compile(
    r"""^\s*(?:[-•\u2022]\s*)?      # optional bullet
        (?P<name>[A-Z][A-Za-z'.-]+(?:\s[A-Z][A-Za-z'.-]+){1,3})  # Name with 2-4 tokens
        \s*[:–—-]\s*                # separator :, –, —, -
        (?P<title>.+?)\s*$          # title/role
    """, re.X)

def extract_roster_from_text(text: str, max_items: int = 14):
    # prefer line-based extraction over sentence-split for rosters
    lines = [ln.strip() for ln in text.splitlines() if ln.strip()]
    out, seen = [], set()
    for ln in lines:
        m = NAME_LINE.match(ln)
        if m:
            name = m.group("name").strip()
            title = m.group("title").strip()
            sig = name.lower()
            if sig in seen: 
                continue
            seen.add(sig)
            # trim overly long titles
            if len(title) > 160:
                title = title[:157] + "…"
            out.append((name, title))
            if len(out) >= max_items:
                break
    return out


In [70]:
# F-4 — compose faculty answer, with per-line citations and the exact instructors page first
def answer_faculty(query: str, k: int = 8):
    hits = retrieve_faculty(query, k=k)
    # prefer the exact instructors page for extraction
    hits_sorted = sorted(hits, key=lambda h: (FACULTY_ALLOW[0] not in (h.get("url","") or ""), ))  # instructors page first

    bullets = []
    used = 0
    for idx, h in enumerate(hits_sorted, 1):
        txt = h.get("text","") or ""
        roster = extract_roster_from_text(txt, max_items=max(8, k))
        if roster:
            for name, title in roster:
                bullets.append(f"- **{name}** — {title} [{idx}]")
                used += 1
                if used >= 14:  # cap list length
                    break
        if used >= 14:
            break

    if not bullets:
        # fallback to your normal answer() which will summarize generatively with citations
        generic, _ = answer(query, k=min(k,6), use_reranker=True)
        return generic, hits

    body = "Here are instructors and staff associated with the **MS in Applied Data Science** program at UChicago:\n\n" + "\n".join(bullets)
    # compact sources list
    src = "\n\nSources:\n" + "\n".join([f"[{i}] {h.get('title','(no title)')} — {h.get('url','')}" for i, h in enumerate(hits_sorted, 1)])
    return body + src, hits_sorted

def ask_faculty(q: str = "Who are the faculty/instructors for the MS in Applied Data Science program?"):
    print("Q:", q, "\n" + "-"*80)
    ans, hits = answer_faculty(q, k=8)
    print(ans)
    return ans, hits


In [74]:
ask(" who are the faculties? Give a list ")

Q:  who are the faculties? Give a list  
--------------------------------------------------------------------------------
The faculty for the MS in Applied Data Science program includes:

1. Greg Green - Senior Instructional Professor and Director of the MS in Applied Data Science Program.
2. Arnab Bose - Associate Senior Instructional Professor and Program Director of MS in Applied Data Science Online Program.
3. Nick Ross - Data Science Clinic Director and Associate Senior Instructional Professor.

For more details, you can visit the official MS-ADS page [5][6]. 

[1] Jobs & Opportunities — https://datascience.uchicago.edu/about/jobs/
[2] Faculty, Leadership, & Staff — https://datascience.uchicago.edu/about/leadership-staff/
[3] Professional Education — https://datascience.uchicago.edu/outreach/industry-partnerships/industry-affiliate-program/professional-education/
[4] DSI Leadership Profiles — https://datascience.uchicago.edu/outreach/industry-partnerships/dsi-leadership-rooted-in-

('The faculty for the MS in Applied Data Science program includes:\n\n1. Greg Green - Senior Instructional Professor and Director of the MS in Applied Data Science Program.\n2. Arnab Bose - Associate Senior Instructional Professor and Program Director of MS in Applied Data Science Online Program.\n3. Nick Ross - Data Science Clinic Director and Associate Senior Instructional Professor.\n\nFor more details, you can visit the official MS-ADS page [5][6].',
 [{'url': 'https://datascience.uchicago.edu/about/jobs/',
   'title': 'Jobs & Opportunities',
   'section': 'about',
   'type': 'generic',
   'date': 'nan',
   'text': 'Rigorous classes, expert instructors, leading-edge technology, and an unparalleled network support your student experience as a full- or part-time learner. Apply Now MS in Computational Analysis and Public Policy (MSCAPP) A rigorous, two-year program offered jointly by the Harris School of Public Policy and the UChicago Department of Computer Science. MSCAPP students wo

In [82]:
ask("What are the core courses in the MS in Applied Data Science program?")

Q: What are the core courses in the MS in Applied Data Science program? 
--------------------------------------------------------------------------------
The core courses in the MS in Applied Data Science program include a comprehensive curriculum that covers statistical modeling, machine learning, data engineering, cloud computing, and AI principles. Students can choose between a 12-course program or an expanded 18-course thesis track, which emphasizes hands-on expertise and prepares them for immediate impact in various industries [3]. 

For more detailed information, you may want to check the official MS-ADS page.

📚 References
[1] Undergraduate Data Science Major — https://datascience.uchicago.edu/education/undergrad-major/
[2] Should You Choose MS in Data Science or MS in Applied Data Science at UChicago? — https://datascience.uchicago.edu/news/should-you-choose-ms-in-data-science-or-ms-in-applied-data-science-at-uchicago/
[3] Info Session – MS in Applied Data Science (In-Person Pr

('The core courses in the MS in Applied Data Science program include a comprehensive curriculum that covers statistical modeling, machine learning, data engineering, cloud computing, and AI principles. Students can choose between a 12-course program or an expanded 18-course thesis track, which emphasizes hands-on expertise and prepares them for immediate impact in various industries [3]. \n\nFor more detailed information, you may want to check the official MS-ADS page.\n\n📚 References\n[1] Undergraduate Data Science Major — https://datascience.uchicago.edu/education/undergrad-major/\n[2] Should You Choose MS in Data Science or MS in Applied Data Science at UChicago? — https://datascience.uchicago.edu/news/should-you-choose-ms-in-data-science-or-ms-in-applied-data-science-at-uchicago/\n[3] Info Session – MS in Applied Data Science (In-Person Program) — https://datascience.uchicago.edu/events/info-session-ms-in-applied-data-science-in-person-program/\n[4] Jobs & Opportunities — https://d